In [96]:
import os
import subprocess
home = os.path.expanduser('~')

os.environ['ROS_DOMAIN_ID']="4"
print(os.environ['ROS_DOMAIN_ID'])
print(os.environ['FASTRTPS_DEFAULT_PROFILES_FILE'])
print(os.environ['RMW_IMPLEMENTATION'])

'4'

In [97]:
list=subprocess.run(['ros2','topic','list'],capture_output=True)

In [98]:
list.stdout.decode().split('\n')[0]

'/Remy/battery_state'

In [82]:
! printenv | grep 'ROS'

ROS_VERSION=2
ROS_PYTHON_VERSION=3
ROS_DOMAIN_ID=4
ROS_LOCALHOST_ONLY=0
ROS_DISTRO=galactic


In [89]:
import time
for i in range(17):
    os.environ['ROS_DOMAIN_ID']=str(i)
    print(os.environ['ROS_DOMAIN_ID'])
    time.sleep(0.1)
    list = subprocess.run(['ros2','topic','list'],capture_output=True)
    print(list.stdout.decode().split('\n')[0])

0
/parameter_events
1
/parameter_events
2
/parameter_events
3
/parameter_events
4
/Remy/battery_state
5
/parameter_events
6
/parameter_events
7
/parameter_events
8
/parameter_events
9
/parameter_events
10
/parameter_events
11
/parameter_events
12
/parameter_events
13
/parameter_events
14
/parameter_events
15
/Dot/battery_state
16
/parameter_events


If that did not work - lets check the RMW implementation - it should be fastrtps

In [44]:
print(os.environ['RMW_IMPLEMENTATION'])
os.environ['RMW_IMPLEMENTATION']= 'rmw_fastrtps_cpp'
print(os.environ['RMW_IMPLEMENTATION'])

rmw_fastrtps_cpp
rmw_fastrtps_cpp


If that is fine, then lets check the multicast

In [45]:
print(os.environ['FASTRTPS_DEFAULT_PROFILES_FILE'])

/home/parallels/Documents/disable_multicast.xml


It should read the same as the file below if there is no response, then run this line - and it will set it to a file in your Documents folder

In [46]:
os.environ['FASTRTPS_DEFAULT_PROFILES_FILE'] = home +'/Documents/disable_multicast.xml'

Next you need to save the list of IP addresses - MAKE SURE to change the file name to the path above

In [ ]:
%%writefile /home/parallels/Documents/disable_multicast.xml

<?xml version="1.0" encoding="UTF-8" ?>
<profiles xmlns="http://www.eprosima.com/XMLSchemas/fastRTPS_Profiles">
   <participant profile_name="unicast_connection" is_default_profile="true">
       <rtps>
           <builtin>
               <metatrafficUnicastLocatorList>
                   <locator/>
               </metatrafficUnicastLocatorList>
               <initialPeersList>
                    <locator>
                       <udpv4>
                           <address>127.0.0.1</address>
                           <!--Localhost -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.156.115 </address>
                           <!--Syndrome -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.152.174</address>
                           <!--Dory -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.157.95</address>
                           <!--Nemo -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.150.128</address>
                           <!--Remy -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.146.11</address>
                           <!--EdnaMode  -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.157.182</address>
                           <!--Woody  -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.156.217</address>
                           <!--Buzz -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.154.102</address>
                           <!--MikeWazowski -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.147.32</address>
                           <!--LightningMcQueen  -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.144.147</address>
                           <!--Mater -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.153.15</address>
                           <!--Dash -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.150.175</address>
                           <!--ElastaGirl -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.92.65</address>
                           <!--Pig -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.153.159</address>
                           <!--PotatoHead -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.148.45</address>
                           <!--Dot -->
                       </udpv4>
                   </locator>
                    <locator>
                       <udpv4>
                           <address>10.245.154.21</address>
                           <!--Sully -->
                       </udpv4>
                   </locator>
               </initialPeersList>
           </builtin>
       </rtps>
   </participant>
</profiles>
<!--ros2 daemon stop 🤔️ source ~/.bashrc 🤔️ ros2 topic list-->

Stop the daemon and start it up again

In [100]:
!ros2 daemon stop

The daemon has been stopped


Then restart everything by asking for a topic list - sometimes this takes up to 1-2 min to update

In [101]:
!export ROS_DOMAIN_ID=4

In [107]:
!ros2 topic list

/Remy/battery_state
/Remy/cmd_audio
/Remy/cmd_lightring
/Remy/cmd_vel
/Remy/dock
/Remy/hazard_detection
/Remy/imu
/Remy/interface_buttons
/Remy/ir_intensity
/Remy/ir_opcode
/Remy/kidnap_status
/Remy/mobility_monitor/transition_event
/Remy/mouse
/Remy/odom
/Remy/robot_state/transition_event
/Remy/slip_status
/Remy/static_transform/transition_event
/Remy/stop_status
/Remy/tf
/Remy/tf_static
/Remy/wheel_status
/Remy/wheel_ticks
/Remy/wheel_vels
/parameter_events
/rosout
